In [2]:
#!/usr/bin/python3
import numpy as np
import transforms3d


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
import csv
import os

from matplotlib import cm

# Append the root dir
import sys, roslib, os
lias_anp_dir = roslib.packages.get_pkg_dir('lias_anp')
scripts_dir = os.path.abspath(os.path.join(lias_anp_dir, 'scripts'))
sys.path.append(scripts_dir)
from utils.sonar_data_processor import SonarDataReader
from utils.match_pairs import get_match_pairs
from anp.anp_alg import AnPAlgorithm
from tri.tri import ANRS, GTRS, gradient_descent
from utils.pose2matrix import pose_to_transform_matrix, ros_pose_to_transform_matrix, transform_matrix_to_ros_pose
from utils.coordinate_system_transform import coordinate_transform_Pose, coordinate_transform_Pose_back, coordinate_transform_pt, coordinate_transform_pt_back
from utils.transformation_matrix_add_noise import add_noise_to_pose

import yaml
yaml_file_path = os.path.join(lias_anp_dir, 'yaml/odom.yaml')
with open(yaml_file_path, 'r') as file:
    params = yaml.safe_load(file)
    RECONSTRUCTION_ERROR_THRESHOLD = params['RECONSTRUCTION_ERROR_THRESHOLD']
    RECORD = params['RECORD']
    DATA_PATH = params['data_path']
    ANP_METHOD = params['ANP_METHOD']

DATA_PATH

'/data/orientation_along_tangent_direction/Rho_noise0.00001theta_noise0.00001/circle/sonar_data.csv'

In [3]:
sonar_data_dir = str(lias_anp_dir) + DATA_PATH
reord_dir = str(lias_anp_dir) + "/record/" + ANP_METHOD
reader = SonarDataReader(filepath = sonar_data_dir)
reader.read_data()
data = reader.get_data()


# anp_algorithm = AnPAlgorithm(ANP_METHOD)

# initialize
start_index = 1
T0 = ros_pose_to_transform_matrix(data[start_index]['pose'])
T1 = ros_pose_to_transform_matrix(data[start_index+1]['pose']) # This is what we need to initialize
# T0 = add_noise_to_pose(T0, rotation_noise_std=0.00001, translation_noise_std=0.0001)
# T1 = add_noise_to_pose(T1, rotation_noise_std=0.00001, translation_noise_std=0.0001)

theta_Rho0 = data[start_index]['si_q_theta_Rho']
pts_indice0 = data[start_index]['pts_indice']
theta_Rho1 = data[start_index+1]['si_q_theta_Rho']
pts_indice1 = data[start_index+1]['pts_indice']
theta_Rho, theta_Rho_prime, common_indices = get_match_pairs(theta_Rho0, pts_indice0, theta_Rho1, pts_indice1)



In [4]:

def ANRS(T_matrix, theta_Rho, theta_Rho_prime):
    """_summary_

    Args:
        T_matrix (_type_): _description_
        theta_Rho (_type_): _description_
        theta_Rho_prime (_type_): _description_

    Returns:
        P_tilde_prime, 
        determinant: DETERMINANT_THRESHOLD is recommended to set to 0.005 to filter out estimated point with high error
        _type_: _description_
    """
    # 将线性方程组写成矩阵形式 A @ P = B
    R_matrix = T_matrix[:3, :3]
    r1 = R_matrix[0, :]
    r2 = R_matrix[1, :]
    t = T_matrix[:3, 3]


    theta = -theta_Rho[0]
    theta_prime = -theta_Rho_prime[0]
    R = theta_Rho[1]  # example value for R
    R_prime = theta_Rho_prime[1] # example value for R'
    
    a1 = np.array([-1, np.tan(theta), 0])
    b1 = 0 
    a2 = np.tan(theta_prime) * r2 - r1
    b2 = t[0] - np.tan(theta_prime) * t[1]
    a3 = t.T @ R_matrix
    b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2

    A = np.vstack([a1, a2, a3])
    b = np.array([b1, b2, b3])
    # print(A)
    # print(b)

    determinant = np.linalg.det(A)
    # if abs(determinant) > DETERMINANT_THRESHOLD:
    # ANRS
    P_o = np.linalg.inv(A) @ b
    norm_P_o = np.linalg.norm(P_o)
    P_hat_o = P_o / norm_P_o

    A_tilde_prime = np.vstack([A, P_hat_o.T])
    b_tilde_prime = np.append(b, R)
    
    P_tilde_prime = np.linalg.inv(A_tilde_prime.T @ A_tilde_prime) @ A_tilde_prime.T @ b_tilde_prime      
    # else:
    #     P_tilde_prime = None
    return P_tilde_prime, determinant 

# Points ground truth
w_P_gt = data[start_index]['w_p']
w_P_gt_indices = [np.where(pts_indice0 == idx)[0][0] for idx in common_indices]
w_P_gt = w_P_gt[w_P_gt_indices] 
# Dictionary to store estimated points in world coordinate system
P_dict = {}
reconstruction_error_list = []
reconstruction_error_list_filtered = []

## TRI!! NEED TO TRANSFORM COORDINATE SYSTEM
T0_tri = coordinate_transform_Pose(T0)
T1_tri = coordinate_transform_Pose(T1)
T_matrix = np.linalg.inv(T1_tri) @ T0_tri
theta_Rho[0]

array([-0.35373211,  4.40943432])

In [ ]:

for i in range(len(theta_Rho)):
    s_P, determinant = ANRS(T_matrix, theta_Rho[i], theta_Rho_prime[i])

    # Transform back to sim coordinate system
    w_P = ( T0_tri @ np.hstack([s_P, 1]) )[:3]
    w_P = coordinate_transform_pt_back( w_P )

    key = common_indices[i]
    difference = np.linalg.norm( w_P_gt[i] - w_P )
    reconstruction_error_list.append(difference)

print(sum(reconstruction_error_list)/len(reconstruction_error_list))
# print(reconstruction_error_list)   
# print(reconstruction_error_list_filtered)   

In [15]:
# Points ground truth
w_P_gt = data[start_index]['w_p']
w_P_gt_indices = [np.where(pts_indice0 == idx)[0][0] for idx in common_indices]
w_P_gt = w_P_gt[w_P_gt_indices] 
# Dictionary to store estimated points in world coordinate system
P_dict = {}
reconstruction_error_list = []
reconstruction_error_list_filtered = []

## TRI!! NEED TO TRANSFORM COORDINATE SYSTEM
T0_tri = coordinate_transform_Pose(T0)
T1_tri = coordinate_transform_Pose(T1)
T_matrix = np.linalg.inv(T1_tri) @ T0_tri
for i in range(len(theta_Rho)):
    s_P, determinant = ANRS(T_matrix, theta_Rho[i], theta_Rho_prime[i])
    # Transform back to sim coordinate system
    w_P = ( T0_tri @ np.hstack([s_P, 1]) )[:3]
    w_P = coordinate_transform_pt_back( w_P )

    key = common_indices[i]
    difference = np.linalg.norm( w_P_gt[i] - w_P )
    reconstruction_error_list.append(difference)
    
    if difference < RECONSTRUCTION_ERROR_THRESHOLD:
        P_dict[key] = w_P
        reconstruction_error_list_filtered.append(difference)
    break
print(reconstruction_error_list)   
print(reconstruction_error_list_filtered)   

[[-1.00000000e+00  4.92286896e-01  0.00000000e+00]
 [-9.99988276e-01  5.02236770e-01  2.16874081e-04]
 [ 3.28649800e-02 -3.56803809e-02 -1.56409727e-03]]
[ 0.          0.05078493 -0.09953466]
[0.011103827687961662]
[0.011103827687961662]


In [17]:
w_P_gt

array([[ 5.68751955e+00, -1.91018105e+00,  6.99972749e-01],
       [ 5.23121881e+00, -1.27144206e+00,  1.70745626e-01],
       [ 5.78667450e+00, -1.61529791e+00,  1.32952547e+00],
       [ 4.27995539e+00,  3.88504565e-01,  1.48389846e-01],
       [ 5.66210842e+00,  2.22324014e+00,  4.33288902e-01],
       [ 6.39459038e+00,  9.44886029e-01,  9.25870299e-01],
       [ 5.39571857e+00, -9.35385823e-01,  7.64190733e-01],
       [ 5.16462421e+00,  1.61509764e+00,  5.76954961e-01],
       [ 6.00825453e+00, -2.38501459e-01,  3.94431025e-01],
       [ 5.70470524e+00,  2.48943424e+00,  1.61083591e+00],
       [ 4.96435022e+00,  2.31796575e+00,  1.20038426e+00],
       [ 4.31123304e+00,  1.52724421e+00,  1.16042435e+00],
       [ 6.26909590e+00,  1.89372346e-01,  1.45200515e+00],
       [ 4.18666077e+00,  1.46283615e+00,  4.90847647e-01],
       [ 3.78225899e+00, -9.00484145e-01,  3.85021955e-01],
       [ 3.29297376e+00, -2.72875249e-01,  9.21835959e-01],
       [ 6.45231915e+00, -2.43982933e-02